## Importing used modules

In [50]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn_pandas import CategoricalImputer
from numpy import array
from sklearn.preprocessing import Imputer,LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

In [9]:
"""imputer for integer"""
def imputer(X,stra):
    X1=X.reshape(-1,1)
    imputer=Imputer(missing_values='NaN',strategy=stra)
    imputer=imputer.fit_transform(X1)
    X1=imputer
    return X1[:,0]
"""imputer for categories"""
def cat_imputer(X):
    imputer=CategoricalImputer()
    imputer=imputer.fit_transform(X)
    X=imputer
    return X

In [10]:
#load training data
train_data=pd.read_csv('train.csv')

In [11]:
##index location 
x=train_data.iloc[:,[2,4,5,6,7,9,11]]
y=train_data.iloc[:,1]

In [12]:
x=x.values
y=y.values
x[0]

array([3, 'male', 22.0, 1, 0, 7.25, 'S'], dtype=object)

In [13]:
x[:,6]=cat_imputer(x[:,6]) ##embark imputing
x[:,2]=imputer(x[:,2],'mean')##age imputing

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0)##splitting training and test data
print('training :',x_train.shape,'test :',x_test.shape)

training : (891, 7) test : (0, 7)


In [15]:
def encode(X):
    #x_train[:,6]=x_train[:,6].astype(str)
    values=X
    #print(values)
    #print(np.shape(values))
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded
    #print(onehot_encoded)

In [16]:
##Sex encode
sex_encoded=encode(x_train[:,1])

In [17]:
x_train=np.column_stack((x_train[:,(0,2,3,4,5,6)],sex_encoded)) 
x_train[0]

array([1, 25.0, 1, 0, 55.4417, 'C', 0.0, 1.0], dtype=object)

In [18]:
##embarke encoding
embarke_encode=encode(x_train[:,5])

In [19]:
x_train=np.column_stack((x_train[:,(0,1,2,3,4,6,7)],embarke_encode))

In [47]:
x_train[:5]

array([[1, 25.0, 1, 0, 55.4417, 0.0, 1.0, 1.0, 0.0, 0.0],
       [3, 21.0, 0, 0, 7.25, 0.0, 1.0, 0.0, 0.0, 1.0],
       [2, 8.0, 0, 2, 26.25, 1.0, 0.0, 0.0, 0.0, 1.0],
       [3, 32.0, 0, 0, 56.4958, 0.0, 1.0, 0.0, 0.0, 1.0],
       [2, 4.0, 2, 1, 39.0, 1.0, 0.0, 0.0, 0.0, 1.0]], dtype=object)

In [51]:
scale = preprocessing.StandardScaler()
x_train[:,[1,4]] = scale.fit_transform(x_train[:,[1,4]])

C:\Users\Abdul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Making the Logistic Regression Ready

In [21]:
def weightInitialization(n_features):
    w = np.random.normal(0, 0.3, (1,n_features))
    b = 0
    return w,b

In [22]:
def sigmoid_activation(z):
    y_pred= 1 / (1 + np.exp(-z))
    return y_pred

In [23]:
def model_optimize(w, b, X, Y):
    """
    w: 1 * number of features
    x: number of examples * number of features
    y:number of examples * 1
    h: 1 * number of examples
    """
    m = X.shape[0]
    lamda = 0.001
    
    #Prediction
    z = np.array( np.dot(w,X.T) + b , dtype=np.float32)
    print("z:", z.shape)
    h = sigmoid_activation(z)
    print("h:", h.shape)
    # TODO: CHeck the cost
    c1 =  (np.dot(Y.T,np.log(h).T))
    c2 = np.dot((1-Y).T , np.log(1-h).T)
    #print("c1:", c1)
    #print("c2:", c2) 
    #print("c1 + c2 :", c1 + c2)
    cost = (-1/m)*((np.dot(Y.T,np.log(h).T)) + np.dot((1-Y).T , np.log(1-h).T))
    print('cost',cost)
    #
    
    #Gradient calculation
    dw = (1/m)*(np.dot((h-Y.T),X))
    print("dw:", dw)
    db = (1/m)*(np.sum(h-Y.T))
    print("db:", db)
    print('m',m)
    
    grads = {"dw": dw, "db": db}
    
    return grads, cost



In [24]:
def update_weights(w,b,X,Y,learning_rate):
    grad,cost = model_optimize(w,b,X,Y)
    w = w - learning_rate * grad['dw']
    b = b - learning_rate * grad['db']
    parameters = {'w':w,'b':b}
    return parameters, cost

In [25]:
def train(w,b,X,Y,learning_rate,num_of_epochs):
    for i in range(num_of_epochs):
        print("x:", X.shape)
        print("y:", Y.shape)
        print("w:", w.shape)
        print("b:", b)
        parameters,cost = update_weights(w,b,X,Y,learning_rate)
        w = parameters["w"]
        b = parameters["b"]
        print("current cost = ", cost)
    return parameters
        

In [26]:
def train(w,b,X,Y,learning_rate,num_of_epochs):
    for i in range(num_of_epochs):
        print("x:", X.shape)
        print("y:", Y.shape)
        print("w:", w.shape)
        print("b:", b)
        parameters,cost = update_weights(w,b,X,Y,learning_rate)
        w = parameters["w"]
        b = parameters["b"]
        print("current cost = ", cost)
    return parameters
        

In [27]:
def predict(parameters,X):
    w = parameters["w"]
    b = parameters["b"]
    
    z = np.array( np.dot(w,X.T) + b , dtype=np.float32)
    print("z:", z.shape)
    h = sigmoid_activation(z)
    return h.T

In [104]:
w,b = weightInitialization(10)
parameters = train(w, b, x_train, y_train, 0.01, 1000)

x: (891, 10)
y: (891,)
w: (1, 10)
b: 0
z: (1, 891)
h: (1, 891)
cost [ 1.06509416]
dw: [[1.0303036130371053 0.016390677536497422 0.26867410751304244
  0.1562262028957457 -0.11646645634672027 0.0103017424225941
  0.3616060981177856 0.04200462460116505 0.038307993500321005
  0.29159522243889363]]
db: 0.37190784054
m 891
current cost =  [ 1.06509416]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.0037190784054
z: (1, 891)
h: (1, 891)
cost [ 1.04990383]
dw: [[1.0159189614115072 0.016633802008683397 0.2665815803605967
  0.15489582983331648 -0.1152830016262068 0.008604206487653499
  0.35747759203049206 0.04119314957414279 0.037862631273858346
  0.2870260176701444]]
db: 0.366081798518
m 891
current cost =  [ 1.04990383]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.00737989639059
z: (1, 891)
h: (1, 891)
cost [ 1.03512541]
dw: [[1.0014563564098244 0.016889916424411927 0.2644595458986516
  0.15354737368496982 -0.11408713691492779 0.0069023766100205965
  0.35333256214422276 0.040379243400094224 0.037412712202

m 891
current cost =  [ 0.65952089]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.132314398482
z: (1, 891)
h: (1, 891)
cost [ 0.65761044]
dw: [[0.33095427474590267 0.036663925825015486 0.1370346574604043
  0.07366359347579991 -0.058576075822205226 -0.07035037948761458
  0.16540608254881153 0.002677644175444923 0.014044364226519996
  0.07833369465923203]]
db: 0.0950557030612
m 891
current cost =  [ 0.65761044]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.133264955513
z: (1, 891)
h: (1, 891)
cost [ 0.65576423]
dw: [[0.32437800461328126 0.03688937290643195 0.1353997207517442
  0.07265864836112686 -0.058066365137809994 -0.07110119474723552
  0.16357920677573595 0.002298769993680361 0.013789813760688946
  0.07638942827413112]]
db: 0.0924780120285
m 891
current cost =  [ 0.65576423]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.134189735633
z: (1, 891)
h: (1, 891)
cost [ 0.65397972]
dw: [[0.31793740642592566 0.03710935744653818 0.1337887894291134
  0.0716692942093251 -0.05756787917704138 -0.0718357673063289


cost [ 0.61084521]
dw: [[0.1322656492927393 0.04245771465733303 0.08258348981004238
  0.04089547310747817 -0.04312471028524804 -0.09207748025488506
  0.10935674723149952 -0.008734446817509388 0.006142313867038081
  0.019871399927085767]]
db: 0.0172792669766
m 891
current cost =  [ 0.61084521]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.156315931572
z: (1, 891)
h: (1, 891)
cost [ 0.61033878]
dw: [[0.12979668141347928 0.042500463318999336 0.08182725096257314
  0.04045700237539615 -0.04292081387635601 -0.09231779782057611
  0.1086342354147255 -0.008872862683669753 0.006041544956791683
  0.019147755321027457]]
db: 0.0163164375941
m 891
current cost =  [ 0.61033878]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.156479095948
z: (1, 891)
h: (1, 891)
cost [ 0.60984219]
dw: [[0.12737748562018614 0.04254087297480134 0.08108371017891684
  0.0400266675100851 -0.04272013305282633 -0.09255164965368415
  0.10792489605720598 -0.009008249349465676 0.005942754807028038
  0.01843874094595947]]
db: 0.0153732464035


current cost =  [ 0.59052486]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.154573297119
z: (1, 891)
h: (1, 891)
cost [ 0.59026954]
dw: [[0.04503011457647138 0.04220005854169421 0.05355158600408488
  0.02503432509072301 -0.03466628568697119 -0.09847475402684325
  0.08206055476408615 -0.0132916135218007 0.0025589232292239517
  -0.005681508970180345]]
db: -0.0164141992628
m 891
current cost =  [ 0.59026954]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.154409155126
z: (1, 891)
h: (1, 891)
cost [ 0.59001569]
dw: [[0.04427306772148034 0.04216455661122575 0.053263648441343604
  0.024894706127218127 -0.0345672622107943 -0.0984885829733946
  0.08178883297609293 -0.01332433279248333 0.0025278272318385135
  -0.005903244436656854]]
db: -0.0166997499973
m 891
current cost =  [ 0.59001569]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.154242157626
z: (1, 891)
h: (1, 891)
cost [ 0.58976328]
dw: [[0.04353041557216752 0.042128452551972596 0.05297993804201668
  0.024757759739654234 -0.03446910418599866 -0.0985004863755

current cost =  [ 0.5786409]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.143262760783
z: (1, 891)
h: (1, 891)
cost [ 0.57843258]
dw: [[0.019837264544367125 0.039804018410248464 0.042818160236350086
  0.02042566264919962 -0.030396837062810687 -0.0973124039159747
  0.07167122227910125 -0.014117273016008064 0.0015351698722368406
  -0.013059078493102233]]
db: -0.0256411816369
m 891
current cost =  [ 0.57843258]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.143006348967
z: (1, 891)
h: (1, 891)
cost [ 0.57822484]
dw: [[0.019556593647442017 0.03975092897786055 0.04267391276479972
  0.02037532385083294 -0.030328117303305067 -0.09726343940270336
  0.07152559600695216 -0.014120720485779307 0.0015240411967139453
  -0.013141164106685841]]
db: -0.0257378433958
m 891
current cost =  [ 0.57822484]
x: (891, 10)
y: (891,)
w: (1, 10)
b: -0.142748970533
z: (1, 891)
h: (1, 891)
cost [ 0.57801766]
dw: [[0.0192812272126961 0.039697692760319564 0.04253132532608898
  0.020325980403206566 -0.030259779689770602 -0.097213

...

db: -0.0237954533863
m 891
current cost =  [ 0.50835639]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.00922256481592
z: (1, 891)
h: (1, 891)
cost [ 0.50828511]
dw: [[0.008148913506907647 0.017370120504158093 0.017038875540632728
  0.01679308431517797 -0.011795834629120842 -0.06120781725905962
  0.03742255366644608 -0.009883995693674527 0.0012048740445831678
  -0.015106141943522175]]
db: -0.0237852635926
m 891
current cost =  [ 0.50828511]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.00946041745185
z: (1, 891)
h: (1, 891)
cost [ 0.50821395]
dw: [[0.008152860867990523 0.01734797731546118 0.017017374887610928
  0.016787893994890078 -0.011778521306077492 -0.06115959164938408
  0.03738450565857251 -0.009878883023320894 0.001204902341737758
  -0.015101105309228438]]
db: -0.0237750859908
m 891
current cost =  [ 0.50821395]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.00969816831176
z: (1, 891)
h: (1, 891)
cost [ 0.50814289]
dw: [[0.008156799875392123 0.017325876427894217 0.016995914857395572
  0.01678269879116

m 891
current cost =  [ 0.50519317]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0200601509708
z: (1, 891)
h: (1, 891)
cost [ 0.50512695]
dw: [[0.008320039604381815 0.016393704937798587 0.016086683028474803
  0.016547917650738684 -0.011028123498484366 -0.05904320006151125
  0.03571457562154926 -0.009657730976606728 0.0012046203903611395
  -0.014875513853716396]]
db: -0.02332862444
m 891
current cost =  [ 0.50512695]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0202934372152
z: (1, 891)
h: (1, 891)
cost [ 0.50506083]
dw: [[0.00832352809833758 0.01637340497275445 0.016066791383104295
  0.016542451282676757 -0.011012062383605276 -0.058997339605616136
  0.03567838660461199 -0.009653007927299484 0.001204580988413022
  -0.014870526062117684]]
db: -0.023318953001
m 891
current cost =  [ 0.50506083]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0205266267452
z: (1, 891)
h: (1, 891)
cost [ 0.50499482]
dw: [[0.008327008790506926 0.01635314267087024 0.01604693253366768
  0.016536978191532003 -0.010996031260097065 -

m 891
current cost =  [ 0.50139857]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0338896472833
z: (1, 891)
h: (1, 891)
cost [ 0.50133828]
dw: [[0.008513376350889838 0.015240317365543822 0.01494981477290024
  0.016211143313299793 -0.010108438678271103 -0.0563774270106914
  0.03361109061340157 -0.009387863304478553 0.001199895785713838
  -0.014578368878525115]]
db: -0.0227663363973
m 891
current cost =  [ 0.50133828]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0341173106473
z: (1, 891)
h: (1, 891)
cost [ 0.50127809]
dw: [[0.00851633479164372 0.015222161590122133 0.014931804236070611
  0.016205393522141627 -0.010093837006421079 -0.056334439117365284
  0.033577173714150754 -0.009383588694830402 0.0011997773959045326
  -0.01457345410428866]]
db: -0.0227572654032
m 891
current cost =  [ 0.50127809]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0343448833013
z: (1, 891)
h: (1, 891)
cost [ 0.50121799]
dw: [[0.008519283105963142 0.015204036508611334 0.014913821879326964
  0.016199640252373436 -0.0100792560885746

h: (1, 891)
cost [ 0.49832727]
dw: [[0.008656619532675588 0.014338930859973252 0.014050981997774641
  0.015907162835969938 -0.009378146106538468 -0.054201676755909436
  0.031894818558770535 -0.009174437484757266 0.0011921859984981226
  -0.014324606710879758]]
db: -0.0223068581971
m 891
current cost =  [ 0.49832727]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0458329797687
z: (1, 891)
h: (1, 891)
cost [ 0.49827167]
dw: [[0.008659168584512674 0.014322419360699236 0.014034424466316147
  0.01590122663814211 -0.009364661894335866 -0.054161001937557955
  0.031862741626807474 -0.009170504096648777 0.0011920069002543236
  -0.014319763114356033]]
db: -0.0222982603108
m 891
current cost =  [ 0.49827167]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0460559623718
z: (1, 891)
h: (1, 891)
cost [ 0.49821615]
dw: [[0.008661713513862405 0.014305938385979414 0.014017894379083825
  0.015895287011876518 -0.009351196735152017 -0.05412036729046258
  0.03183069778344982 -0.009166575608697672 0.0011918276347696582
  -0.

b: 0.0568785412112
z: (1, 891)
h: (1, 891)
cost [ 0.49559505]
dw: [[0.008777742007565419 0.013533691455455403 0.013239298421392003
  0.015600835417390005 -0.008715614090385196 -0.05218074140457728
  0.03030167833622859 -0.008981391319254567 0.0011817114020036127
  -0.014079383151097732]]
db: -0.0218790630683
m 891
current cost =  [ 0.49559505]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0570973318419
z: (1, 891)
h: (1, 891)
cost [ 0.49554351]
dw: [[0.00877994403460478 0.013518627183464768 0.01322402780840025
  0.015594763804632674 -0.008703118177380672 -0.05214218225008176
  0.030271295972344053 -0.008977754449202156 0.0011814796061628194
  -0.014074611434698374]]
db: -0.0218708862777
m 891
current cost =  [ 0.49554351]
x: (891, 10)
y: (891,)
w: (1, 10)
b: 0.0573160407047
z: (1, 891)
h: (1, 891)
cost [ 0.49549206]
dw: [[0.008782136102450551 0.013503589127453865 0.01320878121119691
  0.015588688713265323 -0.008690639080059506 -0.05210366467166563
  0.030240946111721893 -0.008974123014477902 

## Predicting the Labels for the test data using the computed logistic regression parameters

In [105]:
y_pred=predict(parameters,x_train)

z: (1, 891)


In [106]:
y_pred=(y_pred > 0.5).astype(int).reshape(891).tolist()

In [107]:
from sklearn.metrics import confusion_matrix

In [108]:
acc=confusion_matrix(y_train,y_pred)

In [109]:
acc

array([[501,  48],
       [136, 206]], dtype=int64)

In [110]:
accurracy=np.divide(acc[0,0]+acc[1,1],891)

In [111]:
accurracy

0.79349046015712688

In [112]:
#load test data
test_data=pd.read_csv('test.csv')

In [113]:
x_test=test_data.iloc[:,[1,3,4,5,6,8,10]].values

In [114]:
x_test[0]

array([3, 'male', 34.5, 0, 0, 7.8292, 'Q'], dtype=object)

In [115]:
x_test[:,6]=cat_imputer(x_test[:,6])
x_test[:,2]=imputer(x_test[:,2],'mean')
x_test[:,5]=imputer(x_test[:,5],'mean')
x_test[0]

array([3, 'male', 34.5, 0, 0, 7.8292, 'Q'], dtype=object)

In [116]:
##Sex encode
sex_encoded=encode(x_test[:,1])

In [117]:
x_test=np.column_stack((x_test[:,(0,2,3,4,5,6)],sex_encoded)) 
x_test[0]

array([3, 34.5, 0, 0, 7.8292, 'Q', 0.0, 1.0], dtype=object)

In [118]:
embarke_encode=encode(x_test[:,5])

In [119]:
x_test=np.column_stack((x_test[:,(0,1,2,3,4,6,7)],embarke_encode))

In [121]:
x_test[:5]

array([[3, 34.5, 0, 0, 7.8292, 0.0, 1.0, 0.0, 1.0, 0.0],
       [3, 47.0, 1, 0, 7.0, 1.0, 0.0, 0.0, 0.0, 1.0],
       [2, 62.0, 0, 0, 9.6875, 0.0, 1.0, 0.0, 1.0, 0.0],
       [3, 27.0, 0, 0, 8.6625, 0.0, 1.0, 0.0, 0.0, 1.0],
       [3, 22.0, 1, 1, 12.2875, 1.0, 0.0, 0.0, 0.0, 1.0]], dtype=object)

In [123]:
x_test[:,[1,4]]=scale.fit_transform(x_test[:,[1,4]])

C:\Users\Abdul\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [126]:
y_pred=predict(parameters,x_test)

z: (1, 418)


In [128]:
y_pred=(y_pred > 0.5).astype(int).reshape(418).tolist()

In [129]:
import csv

In [130]:
with open('pred.csv','w') as csvfile:    
    writer = csv.writer(csvfile)
# Gives the header name row into csv
    schema = ['PassengerId','Survived']
    writer.writerow([g for g in schema]) 
#Data add in csv file       
    for i in range(418):
        generators = [str(i+892), str(y_pred[i])]
        writer.writerow([g for g in generators])

In [131]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
